# DelaunayInterfaces Julia Examples

This notebook demonstrates the Julia bindings for computing interface surfaces from multicolored point clouds.

In [1]:
using JSON
using GLMakie
using GeometryBasics

# Load the Julia bindings and visualization module
include("../julia/src/DelaunayInterfaces.jl")
using .DelaunayInterfaces

include("../julia/src/visualization.jl")

println("DelaunayInterfaces Julia bindings loaded")

DelaunayInterfaces Julia bindings loaded


## 1. Random Point Cloud Example

In [2]:
using Random
Random.seed!(42)

n_points = 20
n_colors = 3

rand_points = [rand(3) .* 5 for _ in 1:n_points]
rand_colors = rand(1:n_colors, n_points)

# Compute unweighted Delaunay interface
rand_surface = InterfaceSurface(rand_points, rand_colors)

num_triangles = count(x -> length(x[1]) == 3, rand_surface.filtration)
println("Random point cloud: $n_points points, $n_colors colors")
println("Interface: $(length(rand_surface.vertices)) vertices, $num_triangles triangles")

display(pointcloud_subdivision_figure(rand_points, rand_colors, rand_surface; title="Random Point Cloud ($n_colors colors)"))

Random point cloud: 20 points, 3 colors
Interface: 249 vertices, 514 triangles


GLMakie.Screen(...)

## 2. Tetrahedron Examples

These examples use the same configurations from the C++ test suite (test_custom_barycenter.cpp).

In [3]:
# Define tetrahedron examples
using LinearAlgebra

# Rotation helper - rotate points around axis by angle
function rotate_points(points, axis, angle)
    axis = normalize(axis)
    K = [0 -axis[3] axis[2]; axis[3] 0 -axis[1]; -axis[2] axis[1] 0]
    R = I + sin(angle) * K + (1 - cos(angle)) * K^2
    [Vector{Float64}(R * p) for p in points]
end

# 1-1-1-1 partition: Unit tetrahedron with 4 distinct colors (slightly rotated)
const UNIT_TETRAHEDRON_BASE = [
    [0.0, 0.0, 0.0],
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0]
]
const UNIT_TETRAHEDRON = rotate_points(UNIT_TETRAHEDRON_BASE, [1.0, 0.5, 0.7], π/6)

const EXAMPLE_1111 = (
    name = "1-1-1-1 Partition",
    points = UNIT_TETRAHEDRON,
    colors = [1, 2, 3, 4],
    expected_vertices = 11,
    expected_triangles = 12
)

# Shared points for 2-2 and 2-1-1 examples
const SHARED_TETRAHEDRON = [[-1.0, -1.0, 0.0], [1.0, -1.0, 0.0], [0.0, 1.0, -1.0], [0.0, 1.0, 1.0]]

# 2-2 partition: Two points at y=-1 (color 1), two at y=1 (color 2)
const EXAMPLE_22 = (
    name = "2-2 Partition",
    points = SHARED_TETRAHEDRON,
    colors = [1, 1, 2, 2],
    expected_vertices = 9,
    expected_triangles = 8
)

# 2-1-1 partition: Same points, different coloring
const EXAMPLE_211 = (
    name = "2-1-1 Partition",
    points = SHARED_TETRAHEDRON,
    colors = [1, 1, 2, 3],
    expected_vertices = 10,
    expected_triangles = 10
)

# 3-1 partition: Equilateral triangle at y=-1, apex at y=1
const EXAMPLE_31 = (
    name = "3-1 Partition",
    points = let r = 1.0
        [[r, -1.0, 0.0], 
         [-r/2, -1.0, r*sqrt(3)/2], 
         [-r/2, -1.0, -r*sqrt(3)/2], 
         [0.0, 1.0, 0.0]]
    end,
    colors = [1, 1, 1, 2],
    expected_vertices = 7,
    expected_triangles = 6
)

# Verify all tetrahedron examples
for example in [EXAMPLE_1111, EXAMPLE_22, EXAMPLE_211, EXAMPLE_31]
    surface = InterfaceSurface(example.points, example.colors)
    num_verts = length(surface.vertices)
    num_tris = count(x -> length(x[1]) == 3, surface.filtration)
    
    println("\n$(example.name):")
    println("  Colors: $(example.colors)")
    println("  Vertices: $num_verts (expected $(example.expected_vertices))")
    println("  Triangles: $num_tris (expected $(example.expected_triangles))")
end


1-1-1-1 Partition:
  Colors: [1, 2, 3, 4]
  Vertices: 11 (expected 11)
  Triangles: 12 (expected 12)

2-2 Partition:
  Colors: [1, 1, 2, 2]
  Vertices: 9 (expected 9)
  Triangles: 8 (expected 8)

2-1-1 Partition:
  Colors: [1, 1, 2, 3]
  Vertices: 10 (expected 10)
  Triangles: 10 (expected 10)

3-1 Partition:
  Colors: [1, 1, 1, 2]
  Vertices: 7 (expected 7)
  Triangles: 6 (expected 6)


In [4]:
# Visualize 1-1-1-1 partition
surface = InterfaceSurface(EXAMPLE_1111.points, EXAMPLE_1111.colors)
display(tetrahedron_subdivision_figure(EXAMPLE_1111.points, EXAMPLE_1111.colors, surface; 
    title=EXAMPLE_1111.name))

GLMakie.Screen(...)

In [5]:
# Visualize 2-2 partition
surface = InterfaceSurface(EXAMPLE_22.points, EXAMPLE_22.colors)
display(tetrahedron_subdivision_figure(EXAMPLE_22.points, EXAMPLE_22.colors, surface; 
    title=EXAMPLE_22.name))

GLMakie.Screen(...)

In [6]:
# Visualize 2-1-1 partition (same points as 2-2, different colors)
surface = InterfaceSurface(EXAMPLE_211.points, EXAMPLE_211.colors)
display(tetrahedron_subdivision_figure(EXAMPLE_211.points, EXAMPLE_211.colors, surface; 
    title=EXAMPLE_211.name))

GLMakie.Screen(...)

In [7]:
# Visualize 3-1 partition
surface = InterfaceSurface(EXAMPLE_31.points, EXAMPLE_31.colors)
display(tetrahedron_subdivision_figure(EXAMPLE_31.points, EXAMPLE_31.colors, surface; 
    title=EXAMPLE_31.name))

GLMakie.Screen(...)

## 3. Protein Interface Example

Load and visualize a protein dimer interface (4bmg).

In [8]:
# Load protein example
protein_data = JSON.parsefile("../tests/data/ground_truth_4bmg_dimer_alpha.json")

inp = protein_data["input"]
points = [Vector{Float64}(p) for p in inp["points"]]
colors = Vector{Int}(inp["color_labels"])
radii = Vector{Float64}(inp["radii"])

println("Protein: 4bmg_dimer")
println("  Points: $(length(points))")
println("  Unique colors: $(length(unique(colors)))")
println("  Radii range: [$(minimum(radii)), $(maximum(radii))]")

Protein: 4bmg_dimer
  Points: 2259
  Unique colors: 2
  Radii range: [2.82, 3.28]


In [9]:
# Compute interface surface (weighted alpha complex)
surface = InterfaceSurface(points, colors, radii; weighted=true, alpha=true)

num_triangles = count(x -> length(x[1]) == 3, surface.filtration)
println("Interface surface:")
println("  Vertices: $(length(surface.vertices))")
println("  Triangles: $num_triangles")

Interface surface:
  Vertices: 3328
  Triangles: 6098


In [10]:
# Visualize protein interface using the standard visualization style
# Use atom radii (no solvent radius) and Dark2_3 colormap
rs = 1.4 # probe radius used in surface generation
fig = interface_and_point_cloud_figure(
    surface, points, colors, radii.-rs;
    show_wireframe=false,
    point_colormap=:Dark2_3
)
display(fig)

GLMakie.Screen(...)